<hr style="height:2px;">

# Demo: Apply trained CARE model for image transfer in *BCM3D 2.0*

This notebook demonstrates applying a CARE model for a 3D image transfer task, assuming that training was already completed via training.ipynb.  
The trained model is assumed to be located in the folder `models` with the name `my_model`.

In [5]:
#!pip install --user csbdeep

In [4]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE

import os

## Input

In [5]:
modelName = 'model_Cell_Broder_residual_dist3D0410_Shewanella_488_cytosol&membrane'
#modelName = 'BCM3D2_08172022_0107TrainingData_depth3_first32'
modelDir = '/project/GahlmannLab/Yibo/cellDist/Models/ForSegmentation'
outputDir = '/project/GahlmannLab/Yibo/cellDist/ModelOutput/shewan_Dilation2'
# raw data to segment
folder = '/project/GahlmannLab/Yibo/cellDist/Data/Predict/Shewan' 

# add package directory
import sys
sys.path.append("/project/GahlmannLab/Yibo/cellDist/Code/NetworkPredict")

<hr style="height:2px;">

# Load model and input images


In [6]:
# Load model
model = CARE(config = None, name = modelName, 
             basedir = modelDir)
axes ='ZCYX'
#input images

x_images = []
x_filenames = []
#folder = '/project/GahlmannLab/Yibo/cellDist/Data/Predict/Raw' #locate to your raw data
for filename in os.listdir(folder): #import input folder
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        img = imread(os.path.join(folder, filename))
        if img is not None:
            x_images.append(img)
            x_filenames.append(filename)

2022-08-23 17:22:42.625158: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-23 17:22:44.459652: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10788 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:09:00.0, compute capability: 3.7


Loading network weights from 'weights_best.h5'.


## Apply CARE network to input image

Predict the restored image, we suggest you use n_tiles to divide your data if it is too large for memory.

In [7]:
#Save Stored images
from segmentation import segmentation

if not os.path.exists(outputDir): 
    os.mkdir(outputDir)
        
# Predict
img_num = len(x_images)
for k in range(img_num):
    x = x_images[k]
    x = np.stack((x, x), axis=1)
    print('input image size =', x.shape)
    print('input image axes =', axes)

    #%%time
    restored = model.predict(x, axes, n_tiles=(1,1,2,2)) #solve out of memory issue
    print('output image size=', restored.shape)
    print('output image axes =', axes)
    # save NN outputs
    save_tiff_imagej_compatible(outputDir+'/%s_%s' % (model.name, x_filenames[k]), restored, axes)
    # save final BCM3D 2.0 segmentation results
    segmentation(restored, outputDir, x_filenames[k], dilationSize = 2)
    

input image size = (137, 2, 166, 118)
input image axes = ZCYX


2022-08-23 17:22:48.468134: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
100%|██████████| 4/4 [00:02<00:00,  1.50it/s]   


output image size= (137, 2, 166, 118)
output image axes = ZCYX
BgFilter64_11_crop5ms-2_crop.tif


**Note**: *Out of memory* problems during `model.predict` can also indicate that the GPU is used by another process. In particular, shut down the training notebook before running the prediction (you may need to restart this notebook).